In [1]:
import pyedflib
import numpy as np
import xmltodict
import json
import mne
import matplotlib
import pathlib

In [2]:
matplotlib.use('Qt5Agg')

In [3]:
#setting up paths for inputs and outputs
input_path = './input_txt/'
output_path_edf = './output_edf/'
output_path_fif = './output_fif/'

ch_one = '2019-12-17_123934_RawData_Ch1.txt'
ch_two = '2019-12-17_123934_RawData_Ch2.txt'
xml = '2019-12-17_123934.xml'
out_name = '2019-12-17_123934.edf'

ch_one_path = input_path + ch_one
ch_two_path = input_path + ch_two
xml_path = input_path + xml
out_file_path_edf = output_path_edf + out_name
out_file_path_fif = output_path_fif + out_name

In [4]:
#read from .txt and convert data into numpy array
#each dataset is composed of two channels (.txt) and one information doc (.xml)
raw_one = []
raw_two = []

with open(ch_one_path) as f:
    line = f.readline()
    while line:
        raw_one.append(float(line.strip()))
        line = f.readline()
f.close()

with open(ch_two_path) as f:
    line = f.readline()
    while line:
        raw_two.append(float(line.strip()))
        line = f.readline()
f.close()

signal = [np.array(raw_one, dtype=np.float32), np.array(raw_two, dtype=np.float32)]



In [5]:
#read .xml doc and extract needed info
fileptr = open(xml_path, "r")

xml_content = fileptr.read()

my_ordered_dict = xmltodict.parse(xml_content)
dict = json.loads(json.dumps(my_ordered_dict))

sample_rate = eval(dict['RECORD_INFO']['Record']['SamplesFreq'])

In [6]:
#setting up info needed for .edf generation and write .edf file
headers = [{'label':'ch1', 
            'dimension': 'uV',
            'sample_rate': sample_rate,
            'physical_max': 5000,
            "physical_min": -5000,
            'digital_max': 5000,
            'digital_min': -5000,
            'transducer': 'None',
            'prefilter': 'None'},
            {'label':'ch2', 
            'dimension': 'uV',
            'sample_rate': sample_rate,
            'physical_max': 5000,
            "physical_min": -5000,
            'digital_max': 5000,
            'digital_min': -5000,
            'transducer': 'None',
            'prefilter': 'None'}]
with open(out_file_path_edf, 'w') as output:
    print(out_file_path_edf)
    flag = pyedflib.highlevel.write_edf(output.name, signal, headers, header=None, digital=False, file_type=-1, block_size=1)
    print(flag)

./output_edf/2019-12-17_123934.edf
True


In [8]:
#read the newly created .edf using mne
raw=mne.io.read_raw_edf(out_file_path_edf,preload=False)

Extracting EDF parameters from C:\Users\admin\Desktop\work\my_evaluator\output_edf\2019-12-17_123934.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [9]:
raw

<RawEDF | 2019-12-17_123934.edf, 2 x 7746246 (3877.0 s), ~7 kB, data not loaded>

In [ ]:
raw.plot()

Channels marked as bad: none


In [19]:
#create events using mne
#events are equally spaced out for epoch division
new_events = mne.make_fixed_length_events(raw, duration=2.)
event_dict = {'divide':1}
epochs = mne.Epochs(raw,new_events)
epochs.plot()

Not setting metadata
Not setting metadata
1938 matching events found
Setting baseline interval to [-0.2002002002002002, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 1938 events and 1400 original time points ...
1 bad epochs dropped
Loading data for 1937 events and 1400 original time points ...
Loading data for 20 events and 1400 original time points ...


<MNEBrowseFigure size 1919x1016 with 4 Axes>

Loading data for 20 events and 1400 original time points ...
Dropped 0 epochs: 
Channels marked as bad: none


In [21]:
epochs[2].plot()

Loading data for 1 events and 1400 original time points ...
Loading data for 1 events and 1400 original time points ...


<MNEBrowseFigure size 1919x1016 with 4 Axes>

Dropped 0 epochs: 
Channels marked as bad: none
